# Imports

In [1]:
from components.ingestion.synthetic.event_synthetic import EventSynthetic

# Init Component

In [2]:
root_path = "/opt/dev"

general_config = f"{root_path}/pipe_configs/configurations/general_config.ini"
component_config = f"{root_path}/pipe_configs/configurations/event/event_synthetic.ini"

In [3]:
# Start Component
event_synthetic = EventSynthetic(general_config, component_config)
event_synthetic.spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/29 10:05:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Extract

# Transform

In [ ]:
from core.data_objects.bronze.bronze_event_data_object import BronzeEventDataObject
import random

# MVP - Synthetic
spark = event_synthetic.spark
schema = event_synthetic.output_data_objects[BronzeEventDataObject.ID].SCHEMA

n_agents = event_synthetic.config.getint('EventSynthetic', 'n_agents')
n_events = event_synthetic.config.getint('EventSynthetic', 'n_events')
seed = event_synthetic.config.getint('EventSynthetic', 'seed')

sdate = event_synthetic.config.getint('EventSynthetic', 'dates_string')

nrows = n_agents * n_events


data = []
for _ in range(n_agents):
    
    # Empty dataframe
    # df = spark.sparkContext.emptyRDD().toDF(schema)

    # Random user_id
    # random.seed(seed)
    user_id = random.randbytes(32)

    for _ in range(n_events):
        # Random timestamp in path
        random_date = f"{sdate}T{random.randint(0, 23):02}:{random.randint(0, 59):02}:{random.randint(0, 59):02}"
        # random mcc
        random_mcc = random.randint(0, 999)
        # random cell_id
        cell_id = f"{random_mcc:03}" + "".join([str(random.randint(0, 9)) for _ in range(12)])

        # latitude
        latitude = None
        # longitude
        longitude = None
        # loc_error
        loc_error = None
        # Create row
        row = [user_id, random_date, random_mcc, cell_id, latitude, longitude, loc_error]
        # Append row to data
        data.append(row)


df = spark.createDataFrame(data, schema)
df.show()

In [ ]:
event_do = event_synthetic.output_data_objects[BronzeEventDataObject.ID]
sdate = event_synthetic.config.get('EventSynthetic', 'dates_string')
path = f"{event_synthetic.config.get('Paths.Bronze', 'event_data_bronze')}/{sdate}"
print(path)

In [ ]:
event_do.df = df
event_do.write(path)

In [ ]:
temp_output_path = "/opt/data/lakehouse/test_partition"
df.write.format("parquet").partitionBy(["user_id"]).mode("overwrite").save(temp_output_path)

# Load

In [ ]:
import os
import random

bits = random.randbytes(32)
type(bits)

In [ ]:
len(bin(int.from_bytes(bits))[2:])

In [1]:
from pyspark import SparkFiles
SparkFiles.getRootDirectory()

AssertionError: 

In [2]:
SparkFiles.getRootDirectory()

AssertionError: 

In [4]:
import __main__
not(hasattr(__main__, '__file__'))

True